## Installing the enviroment

In [ ]:
!rm -rf master.zip* nvcc4jupyter-master*
!wget https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
!unzip master.zip
!cd nvcc4jupyter-master; ls; python setup.py install

--2022-12-16 04:31:41--  https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master [following]
--2022-12-16 04:31:42--  https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   5.08K  --.-KB/s    in 0.002s  

2022-12-16 04:31:42 (2.08 MB/s) - ‘master.zip’ saved [5201]

Archive:  master.zip
aac710a35f52bb78ab34d2e52517237941399eff
   creating: nvcc4jupyter-master/
 extracting: nvc

In [ ]:
# Crea carpeta src
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


## Generamos el codigo C++

In [ ]:
!mkdir cuda
!rm -rf cuda/*

In [13]:
%%writefile cuda/GPU_multi.cu
#include <stdio.h>
#include <algorithm>
#include <iostream>
#include <sys/time.h>
#include <sstream>
#include <limits>
#include <cuda.h>
#include <cuda_runtime.h>

using namespace std;

#define BLOCK_SIZE 512

__global__ void matrixMultiplicationKernel(int* mt1, int* mt2, int* result, int filas1, int columnas1, int columnas2) {

    int fila = blockIdx.y*blockDim.y+threadIdx.y;
    int columna = blockIdx.x*blockDim.x+threadIdx.x;

    //compruebo que no se haya salido de la matriz
    if (fila < filas1 && columna < columnas2) {
        int tmpSum = 0;
        // cada thread se encarga de un elemento
        for (int i = 0; i < columnas1; i++) {
            tmpSum += mt1[fila * columnas1 + i] * mt2[i * columnas2 + columna];
        }
        //guardamos la suma total en su posición
      result[fila * columnas2 + columna] = tmpSum;
    }
}

int *multiplicaMatrices(int *mat1, int *mat2, int filas1, int comp, int columnas2){
  int *matRes = (int *)calloc(filas1 * columnas2, sizeof(int));

  for (int i = 0; i < filas1; i++){
    for (int j = 0; j < columnas2; j++){
      for (int k = 0; k < comp; k++){
        matRes[i * columnas2 + j] += mat1[i * comp + k] * mat2[k * comp + j];
      }
    }
  }
  return matRes;
}


int main(){
  int numMatrices;
  struct timeval ini, fin;

  do {
      //cout << "Introduce el número de matrices (minimo 2): ";
      cin >> numMatrices;
      cin.clear();
      cin.ignore(numeric_limits<streamsize>::max(),'\n');
  }while (numMatrices < 2);
  
  //matriz en cada fila tiene el vector de la matriz de ese índice
  int **matrices = (int **)calloc(numMatrices, sizeof(int *));
  int *arrayFilas = (int *)calloc(numMatrices, sizeof(int *));
  int *arrayColumnas = (int *)calloc(numMatrices, sizeof(int *));

  for (int ind_matric=0; ind_matric!=numMatrices; ind_matric++){
      int filas, columnas;
      do{
        //cout << "Introduce el número de filas y columnas recuerda que deben ser mayor que 0([filas] [columnas]): ";
        cin >> filas >> columnas;
        cin.clear();
        cin.ignore(numeric_limits<streamsize>::max(),'\n');
      }while(filas<=0 || columnas<=0);
      
      if(ind_matric!=0){
        if(filas!=arrayColumnas[ind_matric-1]){
          cout<<"No es posible esa multiplicación de matrices";
          return 1;
        }
      }
      
      arrayFilas[ind_matric]=filas;
      arrayColumnas[ind_matric]=columnas;
      
      int *matriz = (int *)calloc(filas * columnas, sizeof(int));
      
      for(int fila=0; fila!=filas; fila++){
        //cout<<"Introduzca la fila: ";
        string strFila;
        getline(cin, strFila);
        cin.clear();
        //int *arr = (int *)calloc(tam, sizeof(int)); //fila con los números
        
        istringstream ss(strFila); //hacemos un string stream para pasar de string a int
        string del;
   
        for(int aux=0; getline(ss, del, ' ') && aux!=columnas; aux++) { //con el getline separamos hasta el espacio
            matriz[aux + fila * columnas] = stoi(del); //convertimos de string a número
        }
      }

      matrices[ind_matric]=matriz;
  }
  
  float timeCPU = 0.0;
  int *resMatrix = matrices[0];
  for (int i = 1; i < numMatrices; i++){
    gettimeofday(&ini,NULL);
    resMatrix = multiplicaMatrices(resMatrix, matrices[i], arrayFilas[0], arrayColumnas[i-1], arrayColumnas[i]);
    gettimeofday(&fin,NULL);
    timeCPU += ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0;
  }
  printf("CPU = %f\n", timeCPU);

  int *d_in1, *d_in2, *d_out;
  int *out, imparFlag;
  float timeGPU1 = 0.0;

  resMatrix = matrices[0];
  for (int i = 1; i < numMatrices; i++){

    // Alloc space for device copies
    cudaMalloc((void **)&d_in1, arrayFilas[0]*arrayColumnas[i-1]*sizeof(int));
    cudaMalloc((void **)&d_in2, arrayFilas[i]*arrayColumnas[i]*sizeof(int));

    //resultado que tendrá filas del primero y columnas del segundo
    cudaMalloc((void **)&d_out, arrayFilas[0]*arrayColumnas[i]*sizeof(int));

    // Copy to device
    cudaMemcpy(d_in1, resMatrix, arrayFilas[0]*arrayColumnas[i-1]*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_in2, matrices[i], arrayFilas[i]*arrayColumnas[i]*sizeof(int), cudaMemcpyHostToDevice);

    //inicializamos la salida
    out = (int *)calloc(arrayFilas[0]*arrayColumnas[i], sizeof(int));

    gettimeofday(&ini,NULL);
    matrixMultiplicationKernel<<<arrayFilas[0], arrayColumnas[i-1]>>>(d_in1, d_in2, d_out, arrayFilas[0], arrayColumnas[i-1], arrayColumnas[i]);
    gettimeofday(&fin,NULL);
    timeGPU1 += ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0;

    //Copiamos el resultado al host
    cudaMemcpy(out, d_out,  arrayFilas[0]*arrayColumnas[i]*sizeof(int), cudaMemcpyDeviceToHost);
    resMatrix = out;

    cudaFree(d_in1); cudaFree(d_in2); cudaFree(d_out);
  }
  printf("GPU1 = %f\n", timeGPU1);

  /*
  for(int i=0; i!=arrayFilas[0]; i++){
    for(int j=0; j!=arrayColumnas[numMatrices-1]; j++){
      printf("%d " , resMatrix[i*arrayColumnas[numMatrices-1]+j]);
    }
    cout << "\n";
  }
  cout << "\n";
  */
  
  float timeGPU2 = 0.0;
  for ( ; numMatrices > 1; numMatrices/=2){
    if (numMatrices % 2 == 0)
      imparFlag = 0;
    else
      imparFlag = 1;
    for (int index = imparFlag; index < numMatrices; index += 2){

      // Alloc space for device copies
      cudaMalloc((void **)&d_in1, arrayFilas[index]*arrayColumnas[index]*sizeof(int));
      cudaMalloc((void **)&d_in2, arrayFilas[index+1]*arrayColumnas[index+1]*sizeof(int));

      //resultado que tendrá filas del primero y columnas del segundo
      cudaMalloc((void **)&d_out, arrayFilas[index]*arrayColumnas[index+1]*sizeof(int));

      //inicializamos la salida
      out = (int *)calloc(arrayFilas[index]*arrayColumnas[index+1], sizeof(int));

      // Copy to device
      cudaMemcpy(d_in1, matrices[index], arrayFilas[index]*arrayColumnas[index]*sizeof(int), cudaMemcpyHostToDevice);
      cudaMemcpy(d_in2, matrices[index+1], arrayFilas[index]*arrayColumnas[index+1]*sizeof(int), cudaMemcpyHostToDevice);

      //int n_blocks = ceil(arrayFilas[index]*arrayColumnas[index+1]/BLOCK_SIZE);
      //dim3 threadsPerBlock (BLOCK_SIZE, BLOCK_SIZE);
      //dim3 blocksPerGrid (n_blocks,n_blocks);

      gettimeofday(&ini,NULL);
      //llamo a la multiplicación de matrices
      matrixMultiplicationKernel<<<arrayFilas[index], arrayColumnas[index+1]>>>(d_in1, d_in2, d_out, arrayFilas[index], arrayColumnas[index], arrayColumnas[index+1]);
      gettimeofday(&fin,NULL);

      timeGPU2 += ((fin.tv_sec*1000000+fin.tv_usec)-(ini.tv_sec*1000000+ini.tv_usec))*1.0/1000000.0;

      //Copiamos el resultado al host
      cudaMemcpy(out, d_out,  arrayFilas[index]*arrayColumnas[index+1]*sizeof(int), cudaMemcpyDeviceToHost);

      free(matrices[index]); free(matrices[index+1]);
      cudaFree(d_in1); cudaFree(d_in2); cudaFree(d_out);

      matrices[index/2+imparFlag] = out;
      arrayFilas[index/2+imparFlag] = arrayFilas[index];
      arrayColumnas[index/2+imparFlag] = arrayColumnas[index+1];
    }
    numMatrices += imparFlag;
  }

  printf("GPU2 = %f\n", timeGPU2);
  
  /*
  for(int i=0; i!=arrayFilas[0]; i++){
    for(int j=0; j!=arrayColumnas[0]; j++){
      printf("%d " , matrices[0][i*arrayColumnas[0]+j]);
    }
    cout << "\n";
  }
  cout << "\n";
  */

  return 0;
}

Overwriting cuda/GPU_multi.cu


In [3]:
from random import randint

f = open("cuda/entrada.txt", "w")

numMatrices = 1000
f.write(str(numMatrices) + "\n")

exit()

dimensions = [randint(10, 50) for _ in range(numMatrices + 1)]
for i in range(numMatrices):
  f.write(str(dimensions[i]) + " " + str(dimensions[i+1]) + "\n")
  for _ in range(dimensions[i]):
    row = ""
    for _ in range(dimensions[i+1] - 1):
      row += str(randint(0, 5)) + " "
    row += str(randint(0, 5)) + "\n"
    f.write(row)

f.close()

In [1]:
!nvcc cuda/*.cu -o cuda/program
!./cuda/program < cuda/entrada.txt
!rm -rf ./cuda/program

CPU = 0.392700
GPU1 = 0.006264
GPU2 = 0.007854
